# 介紹

在本課中，我們將了解如何構建能夠學習深度神經網絡著名的複雜關係類型的神經網絡。

這裡的關鍵思想是*模塊化(modularity)*，從更簡單的特徵單元構建一個複雜的網絡。我們已經看到了線性單元如何計算線性函數 —— 現在我們將看到如何組合和修改這些單個單元來模擬更複雜的關係。

# 層(Layers)

神經網絡通常將其神經元組織成**層**。當我們將具有一組公共輸入的線性單元收集在一起時，我們會得到一個 **dense** 層。

<figure style="padding: 1em;">
<img src="https://i.imgur.com/2MA4iMV.png" width="300" alt="輸入層中的三個圓圈堆疊，連接到密集層中的兩個圓圈。">
<figcaption style="textalign: center; font-style: italic"><center>兩個線性單元的密集層，接收兩個輸入和一個偏差。
</center></figcaption>
</figure>

我們可以將神經網絡中的每一層視為執行某種相對簡單的轉換。通過深層堆疊，神經網絡可以以越來越複雜的方式轉換其輸入。在訓練有素的神經網絡中，每一層都是一個轉換，讓我們更接近解決方案。

<blockquote style="margin-right:auto; margin-left:auto; background-color: #000000; padding: 1em; margin:24px;">
    <strong>多種層</strong><br>
Keras 中的“層”是一種非常通用的東西。本質上，層可以是任何類型的<em>數據轉換</em>。許多層，例如 <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D">卷積</a> 和 <a href="https://www. tensorflow.org/api_docs/python/tf/keras/layers/RNN">循環</a>層，通過使用神經元轉換數據，主要區別在於它們形成的連接模式。其他雖然用於 <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding">特徵工程</a> 或只是 <a href="https:// www.tensorflow.org/api_docs/python/tf/keras/layers/Add">簡單算術</a>。有很多層可供探索——<a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers">了解一下</a>！
</blockquote>

# 激活函數

然而，事實證明，中間沒有任何東西的兩個密集層本身並不比單個密集層好。**密集的層本身永遠無法將我們帶出線和平面的世界**。我們需要的是*非線性*。我們需要的是激活函數。

<figure style="padding: 1em;">
<img src="https://i.imgur.com/OLSUEYT.png" width="400" alt=" ">
<figcaption style="textalign: center; font-style: italic"><center>沒有激活函數，神經網絡只能學習線性關係。為了擬合曲線，我們需要使用激活函數。
</center></figcaption>
</figure>

**激活函數**只是我們應用於每個層的輸出（它的*激活*）的一些函數。最常見的是 *rectifier* 函數 $max(0, x)$。

<figure style="padding: 1em;">
<img src="https://i.imgur.com/aeIyAlF.png" width="400" alt="整流函數圖。線 y=x when x>0 and y=0 when x< 0，製作像'_/'這樣的'鉸鏈'形狀。">
<figcaption style="textalign: center; font-style: italic"><center>
</center></figcaption>
</figure>

rectifier 函數有一個圖形，它是一條負部分“整流(rectified)”為零的線。將該函數應用於神經元的輸出將使數據產生*彎曲*，使我們遠離簡單的線條。

當我們將整流器連接到線性單元時，我們得到一個 **rectified linear unit** 或 **ReLU**。 （出於這個原因，通常將 rectifier 函數稱為“ReLU 函數”。）將 ReLU 激活應用於線性單元意味著輸出變為 `max(0, w * x + b)`，我們可以將其繪製為圖表如：

<figure style="padding: 1em;">
<img src="https://i.imgur.com/eFry7Yu.png" width="400" alt="單個 ReLU 的圖。就像一個線性單元，但我們現在有一個鉸鏈(hinge shape)'_/'。">
<figcaption style="textalign: center; font-style: italic"><center>校正線性單位。
</center></figcaption>
</figure>

# 堆疊密集層(Stacking Dense Layers) #

現在我們有了一些非線性，讓我們看看如何堆疊層來獲得複雜的數據轉換。

<figure style="padding: 1em;">
<img src="https://i.imgur.com/Y5iwFQZ.png" width="450" alt="一個輸入層、兩個隱藏層和一個最終線性層。">
<figcaption style="textalign: center; font-style: italic"><center>一疊密集層構成一個“全連接”網絡。
</center></figcaption>
</figure>

**輸出層之前的層**有時被稱為 **hidden** 因為我們從來沒有直接看到它們的輸出。

現在，請注意最終（輸出）層是一個**線性單元**（意思是，沒有激活函數）。這使得這個網絡適用於迴歸任務，我們試圖預測一些任意數值。其他任務（如分類）可能**需要輸出激活函數**。

## 建立序列模型

我們一直使用的`序列`模型將按照從第一到最後的序列連接一系列層：第一層獲取輸入，最後一層產生輸出。這將創建上圖中的模型：

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    # 隱藏層，用relu激活
    layers.Dense(units=4, activation='relu', input_shape=[2]),
    layers.Dense(units=3, activation='relu'),
    # 線性輸出層
    layers.Dense(units=1),
])

確保將所有層一起傳遞到一個列表中，例如 `[layer, layer, layer, ...]`，而不是作為單獨的參數。 要將激活函數添加到層，只需在 `activation` 參數中給出其名稱。

# EXERCISE

現在，[**為 *Concrete* 數據集創建一個深度神經網絡**](https://www.kaggle.com/kernels/fork/11887344)。